[Python for Finance](https://github.com/yhilpisch/py4fi)

$$
C(S,K,\tau,r,\sigma)=SN(d_1)-Ke^{-r\tau}N(d_2)
$$
$$
d_1=\frac{\log\frac{S}{K}+\left(r+\frac{\sigma^2}{2}\right)\tau}{\sigma\sqrt{\tau}}
$$
$$
d_2=\frac{\log\frac{S}{K}+\left(r-\frac{\sigma^2}{2}\right)\tau}{\sigma\sqrt{\tau}}
$$

In [1]:
def C(S, K, tau, r, sigma):
    import numpy as np
    import scipy.stats as stats
    d1 = (np.log(S/K) + (r+sigma**2/2)*tau) / (sigma*np.sqrt(tau))
    d2 = (np.log(S/K) + (r-sigma**2/2)*tau) / (sigma*np.sqrt(tau))
    N = stats.norm().cdf 
    return S*N(d1) - K*np.exp(-r*tau)*N(d2)

S0 = 100
K = 105
tau = 1.0
r = 0.05
sigma = 0.2
print(C(S0, K, tau, r, sigma))

8.021352235143176


$$\begin{array}{lll}
\displaystyle
\frac{dS_t}{S_t}=rdt+\sigma db
&\Rightarrow&
\displaystyle
\frac{S_{t+dt}-S_t}{S_t}=rdt+\sigma db\\
&\Rightarrow&
\displaystyle
S_{t+dt}=S_t\left[1+(rdt+\sigma db)\right]\\
&\Rightarrow&
\displaystyle
S_{t+dt}=S_t e^{\left(r-\frac{1}{2}\sigma^2\right)dt+\sigma db}\\
\end{array}$$

In [ ]:
import time 
import numpy as np
S0 = 100; K = 105; tau = 1.0; r = 0.05; sigma = 0.2  
M = 50  # number of time steps
I = 250000  # number of paths
tic = time.time()

#
# Main Simulation Code
#

toc = time.time() 
print("European Option Value : {}".format(C_simulation))
print("Duration in Seconds   : {}".format(toc-tic))

In [1]:
# Pure Python Simulation (Discrete Version)
import time 
import numpy as np
S0 = 100; K = 105; tau = 1.0; r = 0.05; sigma = 0.2  
M = 50  # number of time steps
I = 250000  # number of paths
tic = time.time()

# Pure Python Simulation (Discrete Version)
paths = []
for i in range(I):
    path = []
    for t in range(M + 1):
        if t == 0:
            path.append(S0)
        else:
            S_t = path[t-1]
            dt = tau / M  
            db = np.sqrt(dt) * np.random.normal()
            path.append(S_t*(1+(r*dt+sigma*db)))
    paths.append(path)
C_simulation = np.exp(-r*tau) * sum([max(path[-1] - K, 0) for path in paths]) / I

toc = time.time() 
print("European Option Value : {}".format(C_simulation))
print("Duration in Seconds   : {}".format(toc-tic))

European Option Value : 8.0065230502632
Duration in Seconds   : 58.639920234680176


In [3]:
# Pure Python Simulation (Continuous Version)
import time 
import numpy as np
S0 = 100; K = 105; tau = 1.0; r = 0.05; sigma = 0.2  
M = 50  # number of time steps
I = 250000  # number of paths
tic = time.time()

# Pure Python Simulation (Continuous Version)
paths = []
for i in range(I):
    path = []
    for t in range(M + 1):
        if t == 0:
            path.append(S0)
        else:
            S_t = path[t-1]
            dt = tau / M 
            db = np.sqrt(dt) * np.random.normal()
            exponent = (r-0.5*sigma**2)*dt + sigma*db
            path.append(S_t*np.exp(exponent))
    paths.append(path)
C_simulation = np.exp(-r*tau) * sum([max(path[-1] - K, 0) for path in paths]) / I

toc = time.time() 
print("European Option Value : {}".format(C_simulation))
print("Duration in Seconds   : {}".format(toc-tic))

European Option Value : 8.042366493086094
Duration in Seconds   : 60.05700898170471


In [2]:
# Vectorization (Continuous Version)
import time 
import numpy as np
S0 = 100; K = 105; tau = 1.0; r = 0.05; sigma = 0.2  
M = 50  # number of time steps
I = 250000  # number of paths
tic = time.time()

# Vectorization (Continuous Version)
S = np.zeros((M + 1, I))
S[0] = S0
for t in range(1, M + 1):
    dt = tau / M 
    db = np.sqrt(dt) * np.random.normal(size=(I,)) 
    exponent = (r-0.5*sigma**2)*dt + sigma*db
    S[t] = S[t - 1] * np.exp(exponent)
C_simulation = np.exp(-r*tau) * np.sum(np.maximum(S[-1] - K, 0)) / I

toc = time.time() 
print("European Option Value : {}".format(C_simulation))
print("Duration in Seconds   : {}".format(toc-tic))

European Option Value : 8.071718856497474
Duration in Seconds   : 0.4447610378265381


In [3]:
# Full Vectorization (Continuous Version)
import time 
import numpy as np
S0 = 100; K = 105; tau = 1.0; r = 0.05; sigma = 0.2  
M = 50  # number of time steps
I = 250000  # number of paths
tic = time.time()

# Full Vectorization (Continuous Version)
dt = tau / M 
db = np.sqrt(dt) * np.random.normal(size=(M+1, I)) 
exponent = (r-0.5*sigma**2)*dt + sigma*db
exponent[0] = 0
exponent = np.cumsum(exponent, axis=0)
S = S0 * np.exp(exponent)
C_simulation = np.exp(-r*tau) * np.sum(np.maximum(S[-1] - K, 0)) / I

toc = time.time() 
print("European Option Value : {}".format(C_simulation))
print("Duration in Seconds   : {}".format(toc-tic))

European Option Value : 7.999981658151316
Duration in Seconds   : 0.592475175857544
